In [1]:
!pip install wrds

  Using cached SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.52
    Uninstalling SQLAlchemy-1.4.52:
      Successfully uninstalled SQLAlchemy-1.4.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-scheduler 2.5.2 requires sqlalchemy~=1.0, but you have sqlalchemy 2.0.31 which is incompatible.


In [22]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

# from plotnine import *
# from mizani.formatters import comma_format, percent_format
from datetime import datetime

In [2]:
import wrds

In [8]:
conn = wrds.Connection()

Enter your WRDS username [sagemaker-user]: jennasparks
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [9]:
conn.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'core',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',

In [10]:
# Extract first 5 obs from crsp.msf

company = conn.get_table(library='crsp', table='msf', obs=5)
company.shape

(5, 21)

In [6]:
company

,gvkey,cik,ein,conm,costat,dlrsn,priusa,prican,prirow,idbflag,...,dldte,stko,fyrc,sic,gsector,ggroup,gind,gsubind,spcindcd,spcseccd
0,001000,None,None,A & E PLASTIK PAK INC,I,09,01,None,None,D,...,1978-06-30,0.0,12.0,3089.0,NaN,NaN,NaN,NaN,325.0,978.0
1,001001,0000723576,73-0934497,A & M FOOD SERVICES INC,I,01,01,None,None,D,...,1986-07-31,0.0,12.0,5812.0,25.0,2530.0,253010.0,25301040.0,420.0,978.0
2,001002,0001306124,None,AAI CORP,I,01,01,None,None,D,...,1977-03-31,0.0,12.0,3825.0,NaN,NaN,NaN,NaN,230.0,940.0
3,001003,0000730052,43-1232041,A.A. IMPORTING CO INC,I,07,01,None,None,D,...,1992-04-30,3.0,1.0,5712.0,25.0,2550.0,255040.0,25504040.0,449.0,976.0
4,001004,0000001750,36-2334820,AAR CORP,A,None,01,None,None,D,...,None,0.0,5.0,5080.0,20.0,2010.0,201010.0,20101010.0,110.0,925.0


In [12]:
# Another table

next_table = conn.get_table(library='crsp', table='stkshares', obs=5)
next_table.shape

(5, 7)

In [13]:
next_table

,permno,shrstartdt,shrenddt,shrout,shrsource,shrfactype,shradrflg
0,10000,1986-01-07,1986-01-30,3680,OBS,NA,N
1,10000,1986-01-31,1986-04-29,3680,OBS,NA,N
2,10000,1986-04-30,1986-07-30,3793,OBS,NA,N
3,10000,1986-07-31,1986-10-30,3793,OBS,NA,N
4,10000,1986-10-31,1986-12-03,3843,OBS,NA,N


In [18]:
start_date = "01/01/2023"
end_date = "12/31/2023"

In [23]:
crsp_monthly_query = (
  "SELECT msf.permno, msf.mthcaldt AS date, "
         "date_trunc('month', msf.mthcaldt)::date AS month, "
         "msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc, "
         "ssih.primaryexch, ssih.siccd "
    "FROM crsp.msf_v2 AS msf "
    "INNER JOIN crsp.stksecurityinfohist AS ssih "
    "ON msf.permno = ssih.permno AND "
       "ssih.secinfostartdt <= msf.mthcaldt AND "
       "msf.mthcaldt <= ssih.secinfoenddt "
   f"WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}' "
          "AND ssih.sharetype = 'NS' "
          "AND ssih.securitytype = 'EQTY' "  
          "AND ssih.securitysubtype = 'COM' " 
          "AND ssih.usincflg = 'Y' " 
          "AND ssih.issuertype in ('ACOR', 'CORP') " 
          "AND ssih.primaryexch in ('N', 'A', 'Q') "
          "AND ssih.conditionaltype in ('RW', 'NW') "
          "AND ssih.tradingstatusflg = 'A'"
)

crsp_monthly = (pd.read_sql_query(
    sql=crsp_monthly_query,
    con=wrds,
    dtype={"permno": int, "siccd": int},
    parse_dates={"date", "month"})
  .assign(shrout=lambda x: x["shrout"]*1000)
)

/tmp/ipykernel_561/542593069.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crsp_monthly = (pd.read_sql_query(


AttributeError: module 'wrds' has no attribute 'cursor'